In [1]:
# url = "https://usaupload.com/72Eb/mnist.zip?download_token=122af6eb9e746e2774d3422cba7775dd9ca2a43aaa3156f4c0b5043723a24823"
# name = url.split("/")[-1]
# fileName = name.split("?")[0]

# !wget $url
# !mv $name $fileName

In [2]:
# %%capture
# !unzip mnist.zip;

In [3]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dense
from keras import optimizers
from keras import backend as K

In [4]:
mySeed = 42
np.random.seed(mySeed)
random.seed(mySeed)
tf.random.set_seed(mySeed)
# torch.manual_seed(mySeed)

In [5]:
def load_mnist_bypath(paths, verbose=-1):
    """ Expect to read images where each class is in a separate directory,
        For example: images of type 0 are in folder 0
    """

    data = list()
    labels = list()
    # loop over the input images
    for (i, imgpath) in enumerate(paths):
        # load the image and extract the class labels
        im_gray = cv2.imread(imgpath, cv2.IMREAD_GRAYSCALE)
        image = np.array(im_gray).flatten()
        label = imgpath.split(os.path.sep)[-2]
        # here the img is scaled to [0, 1] to less the impact of each pixel's brightness
        data.append(image/255)
        labels.append(label)
        # show an update every `verbose` images
        if verbose > 0 and i > 0 and (i+1) % verbose == 0:
            print(f"[INFO] processed {i+1}/{len(paths)}")
            break

    # return a tuple of the data and labels
    return data, labels

In [6]:
# Declare mnist dataset path
img_path = "mnist/trainingSet/trainingSet"
# img_path = "/content/mnist/trainingSet/trainingSet"

# Generate a list of trucks using the list_images function from the paths library
image_paths = list(paths.list_images(img_path))

# load images into arrays
image_list, label_list = load_mnist_bypath(image_paths, verbose=10000)

# Perform the one-hot encoded so we can use the sparse-categorical-entropy loss function
lb = LabelBinarizer()
label_list = lb.fit_transform(label_list)

#split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(image_list, label_list, test_size=0.2, random_state=42)

[INFO] processed 10000/42000


In [7]:
def create_clients(image_list, label_list, num_clients=10, initial="clients"):
    """ return: A dictionary with the customer id as the dictionary key and the value
                will be the data fragment - tuple of images and labels.
        args:
            image_list: a numpy array object with the images
            label_list: list of binarized labels (one-hot encoded)
            num_client: number of customers (clients)
            initials: the prefix of the clients, e.g., clients_1
     """

    # create list of customer names
    client_names = [f"{initial}_{i+1}" for i in range(num_clients)]

    # shuffle the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    # shard the data and split it for each customer
    size = len(data) // num_clients
    shards = [data[i: i+size] for i in range(0, size * num_clients, size)]

    # Check if the fragment number is equal to the number of clients
    assert(len(shards) == len(client_names))

    return {client_names[i]: shards[i] for i in range(len(client_names))}


num_clients = 2
# Create the customers
clients = create_clients(X_train, y_train, num_clients=num_clients, initial="client")

In [8]:
def batch_data(data_shard, b=32):
    """ Receives a piece of data from a client and creates a tensorflow data object in it
        args:
            data_shard: data and labels that make up a customer's data shard
            b: batch size
        return:
            data tensorflow object
    """
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(b)


# Process and collate the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)

# process and group test set
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

In [9]:
test = list(zip([[1, 2], [3, 4], [5, 6], [7, 8]], ["a", "b", "c", "d"]))
print(test)
data, label = zip(*test)
print(data)
print(label)

print((list(data), list(label)), "\n")

dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
test1 = dataset.shuffle(len(label))
print(list(test1.as_numpy_iterator()))
test2 = dataset.shuffle(len(label)).batch(4)
print(list(test2.as_numpy_iterator()))

print("---------")
list(test2)[0][0].shape[0]

tf.data.experimental.cardinality(test1)

[([1, 2], 'a'), ([3, 4], 'b'), ([5, 6], 'c'), ([7, 8], 'd')]
([1, 2], [3, 4], [5, 6], [7, 8])
('a', 'b', 'c', 'd')
([[1, 2], [3, 4], [5, 6], [7, 8]], ['a', 'b', 'c', 'd']) 

[(array([5, 6]), b'c'), (array([3, 4]), b'b'), (array([7, 8]), b'd'), (array([1, 2]), b'a')]
[(array([[1, 2],
       [5, 6],
       [3, 4],
       [7, 8]]), array([b'a', b'c', b'b', b'd'], dtype=object))]
---------


<tf.Tensor: shape=(), dtype=int64, numpy=4>

In [10]:
class MLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(100, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(100))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("softmax"))

        return model

lr = 0.01
comms_round = 1
loss = "categorical_crossentropy"
metrics = ["accuracy"]
# optimizer = SGD(lr=lr, decay=lr/comms_round, momentum=0.9)
# optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=lr, decay=lr/comms_round, momentum=0.9)

# optimizer = optimizers.Adam(learning_rate=lr, decay=lr/comms_round)
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=lr, decay=lr/comms_round)

In [11]:
def weight_scalling_factor(clients_trn_data, client_name, participants):
    """ Calculates the size ratio of a client's local training data
        with all general training data maintained by all customers
    """
    # client_names = list(clients_trn_data.keys())
    # calculate batch size
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    # first calculate total training data across clients
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()
                        for client_name in participants]) * bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() * bs

    return local_count / global_count

In [12]:
def scale_model_weights(weight, scalar):
    """ Scale the model weights """
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])

    return weight_final

In [13]:
def sum_scaled_weights(scaled_weight_list):
    """ Return the sum of the listed scaled weights. O is equivalent to the average weight of the weights """
    avg_grad = list()
    # get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)

    return avg_grad

In [14]:
def test_model(X_test, Y_test, model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    # logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    # print(f"Agregation Round: {comm_round} | global_acc: {acc:.3%} | global_loss: {loss}"")
    print(f"round: {comm_round} | acc: {acc:.3%} | loss: {loss}")

    return acc, loss

In [15]:
def check_local_loss(client, model):
    # Check local loss
    cce_l = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    client_x = np.array([i[0] for i in clients[client]])
    client_y = np.array([i[1] for i in clients[client]])
    logits_l = model.predict(client_x)
    loss_l = cce_l(client_y, logits_l)
    acc_l = accuracy_score(tf.argmax(logits_l, axis=1), tf.argmax(client_y, axis=1))
    print(f"Local accuracy: {acc_l}. Local loss: {loss_l}")

    return acc_l, loss_l

In [16]:
len(X_train)
np.array(X_train).shape[1:]

(784,)

In [29]:
client_select_rate = 0.6
client_select_rate = 1
client_epochs = 1
### Start global template ###

smlp_global = MLP()
global_model = smlp_global.build(784, np.array(y_train).shape[-1])

# Global training loop collection
for comm_round in range(comms_round):

    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()

    # initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    # randomize client data - using keys
    client_names = list(clients_batched.keys())
    random.shuffle(client_names)
    client_select = client_names[0: int(num_clients*client_select_rate)]
    # print(client_select)

    # loop through each client and create a new local model
    for client in client_select:
        smlp_local = MLP()
        local_model = smlp_local.build(784, np.array(y_train).shape[-1])
        local_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

        # set the weight of the local model to the weight of the global model
        local_model.set_weights(global_weights)

        # fit local model with client's data
        local_model.fit(clients_batched[client], epochs=client_epochs, verbose=0)

        # scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client, client_select)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)

        print("test")
        print(local_model.get_weights())
        
        break

    #     # Check local accuracy
    #     # acc_l, loss_l = check_local_loss(client, local_model)

    #     # clear session to free memory after each communication round
    #     K.clear_session()

    # # to get the average over all the local model, we simply take the sum of the scaled weights
    # average_weights = sum_scaled_weights(scaled_local_weight_list)

    # # update global model
    # global_model.set_weights(average_weights)

    # # test global model and print out metrics after each communications round
    # for (X_test, Y_test) in test_batched:
    #     global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

test
[array([[-0.10273433, -0.04704752,  0.00808045, ...,  0.00588168,
        -0.00892175,  0.04568826],
       [-0.08046277, -0.08837207,  0.07953828, ..., -0.01924175,
        -0.0003405 ,  0.05012558],
       [-0.10633839,  0.07703272,  0.14993031, ..., -0.06314261,
         0.07507354, -0.00307084],
       ...,
       [ 0.03414189,  0.04948349, -0.01683494, ..., -0.01540671,
        -0.00967073, -0.07834828],
       [ 0.01090276,  0.02770779, -0.04498865, ...,  0.01563124,
        -0.06267883,  0.01385543],
       [-0.00375625,  0.04419962, -0.04669923, ..., -0.076101  ,
         0.07397996,  0.02152165]], dtype=float32), array([ 0.03723201, -0.04715949, -0.02667499, -0.04260452,  0.03148138,
        0.04119247, -0.04683567, -0.01977017, -0.07218076, -0.01400004,
       -0.10265955,  0.01042398, -0.06595653, -0.01400225, -0.04816522,
       -0.05215771, -0.01774358, -0.03047093, -0.01886008,  0.00966555,
       -0.0549637 ,  0.03881105, -0.07077589, -0.00620759,  0.00185924,
     